In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
import keras
import keras.layers as layers

In [2]:
dte = pd.read_csv("test.csv")
dtr = pd.read_csv("train.csv")

In [3]:
dtr.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

In [4]:
dtr.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


In [5]:
dtr.nunique()

PassengerId     8693
HomePlanet         3
CryoSleep          2
Cabin           6560
Destination        3
Age               80
VIP                2
RoomService     1273
FoodCourt       1507
ShoppingMall    1115
Spa             1327
VRDeck          1306
Name            8473
Transported        2
dtype: int64

In [6]:
dtr

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [7]:
ytr = dtr.pop("Transported").astype("int")
Xtr = dtr
X_train, X_test, y_train, y_test = train_test_split(Xtr, ytr, test_size=0.2, random_state=0)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
4278,4558_01,Europa,False,C/167/S,55 Cancri e,54.0,False,0.0,559.0,0.0,15238.0,2799.0,Wezna Baleful
5971,6326_01,Earth,False,F/1307/P,TRAPPIST-1e,20.0,False,0.0,20.0,1.0,696.0,0.0,Therek Hinetthews
464,0503_02,Mars,False,F/90/S,TRAPPIST-1e,43.0,False,1821.0,0.0,47.0,29.0,0.0,Torms Fone
4475,4757_01,Earth,False,F/896/S,TRAPPIST-1e,24.0,False,185.0,0.0,476.0,1810.0,53.0,Tanley Mirandry
8469,9046_01,Europa,True,C/335/S,55 Cancri e,25.0,False,0.0,0.0,0.0,0.0,0.0,Alphah Cratrave
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4373,4655_01,Europa,True,B/154/P,55 Cancri e,32.0,False,0.0,0.0,0.0,0.0,0.0,Ainkard Seflock
7891,8423_01,Earth,False,F/1620/S,TRAPPIST-1e,22.0,False,0.0,0.0,6.0,0.0,733.0,Analdy Bartez
4859,5185_01,Mars,False,E/330/S,TRAPPIST-1e,29.0,False,523.0,0.0,21.0,4.0,811.0,Cruts Flie
3264,3499_04,Earth,False,G/574/P,TRAPPIST-1e,0.0,False,0.0,0.0,0.0,0.0,0.0,Allene Mccarveymon


In [112]:
new_num = ["TotalExpenses"]
expense_adder = lambda df : df["RoomService"] + df["FoodCourt"] + df["ShoppingMall"] + df["Spa"] + df["VRDeck"]
new_cat = ["CabinSide", "CabinDeck"]


def add_features(df):
    tdf = df.assign(TotalExpenses=expense_adder)
    cabinParsed = df[['Cabin']].fillna("None/-/None").apply(lambda s : pd.Series(s['Cabin'].split('/')[0::2]), axis = 1)
    cabinParsed.columns = ['CabinDeck', 'CabinSide']
    # return pd.concat([tdf, cabinParsed])
    return tdf.join(cabinParsed)

numeric_features = ["Age", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"] + new_num
boolean_features = ["CryoSleep", "VIP"]
categor_features = ["HomePlanet", "Destination"] + new_cat

feature_adder = ColumnTransformer(
    FunctionTransformer(lambda df : df["RoomService"] + df["FoodCourt"] + df["ShoppingMall"] + df["Spa"] + df["VRDeck"]),
    remainder = 'passthrough'
)
numeric_pipeline = Pipeline(steps = [("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())])
categor_pipeline = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers = [
        ("num", numeric_pipeline, numeric_features),
        ("cat", categor_pipeline, categor_features)
    ]
)

class PipelineSpaceship():
    def __init__(self, skp):
        self.skp = skp
        
    def fit(self, df):
        return self.skp.fit(add_features(df))
    
    def transform(self, df):
        return self.skp.transform(add_features(df))
    
ps = PipelineSpaceship(preprocessor)

ps.fit(X_train)
X_train_p = ps.transform(X_train)
X_test_p = ps.transform(X_test)

In [114]:
sklo = LogisticRegression()
sklo.fit(X_train_p, y_train)
sklo.score(X_test_p, y_test)

0.7751581368602645

In [115]:
def build_model(layer_count=2, layer_size=16, loss="binary_crossentropy", activation="relu"):
    model = keras.Sequential([layers.Dense(layer_size, activation=activation)]*layer_count
        + [layers.Dense(1, activation="sigmoid")])
    model.compile(optimizer="rmsprop", loss=loss, metrics=["accuracy"])
    return model

In [116]:
def train(model, patience = 3):
    tensorboard = keras.callbacks.TensorBoard(log_dir="/tmp/tbl")
    # run in console tensorboard --logdir /tmp/tbl
    est = keras.callbacks.EarlyStopping(monitor = "val_accuracy", patience = patience)
    history = model.fit(X_train_p,
                        y_train,
                        epochs=50,
                        batch_size=512,
                        validation_data=(X_test_p, y_test),
                        verbose=0,
                    callbacks = [tensorboard, est])
    for key in ["loss", "accuracy", "val_loss", "val_accuracy"]:
            print(f"{key}[0]: {history.history[key][0]}, {key}[-1]: {history.history[key][-1]}")

In [117]:
train(build_model())

loss[0]: 0.6684522032737732, loss[-1]: 0.42746198177337646
accuracy[0]: 0.6179177165031433, accuracy[-1]: 0.7970951795578003
val_loss[0]: 0.646450936794281, val_loss[-1]: 0.43318507075309753
val_accuracy[0]: 0.6526739597320557, val_accuracy[-1]: 0.7872340679168701


In [118]:
train(build_model(layer_count = 3, layer_size = 64))

loss[0]: 0.7003013491630554, loss[-1]: 0.42703959345817566
accuracy[0]: 0.49870577454566956, accuracy[-1]: 0.796951413154602
val_loss[0]: 0.661185085773468, val_loss[-1]: 0.43553465604782104
val_accuracy[0]: 0.5899942517280579, val_accuracy[-1]: 0.7837837934494019


In [119]:
train(build_model(layer_count = 9, layer_size = 128), patience = 10)

loss[0]: 0.6536716818809509, loss[-1]: 0.3961748778820038
accuracy[0]: 0.641788899898529, accuracy[-1]: 0.8093183636665344
val_loss[0]: 0.6029195189476013, val_loss[-1]: 0.4155769646167755
val_accuracy[0]: 0.7475560903549194, val_accuracy[-1]: 0.8016101121902466


In [120]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier()
gbc.fit(X_train_p, y_train)
gbc.score(X_test_p, y_test)

0.7906843013225991

In [130]:
best = gbc.predict(ps.transform(dte))

In [137]:
result = pd.DataFrame(data={'PassengerId': dte['PassengerId'], 'Transported': best.astype(bool)})
result.to_csv('submission01.csv', index = False)

In [121]:
import xgboost as xgb
# param = {'max_depth':2, 'eta':1, 'objective':'binary:logistic' }
# xtrain = xgb.DMatrix(X_train_p, y_train)
# bst = xgb.train(param, xtrain, 4)
reg = xgb.XGBRegressor(tree_method="gpu_hist")
reg.fit(X_train_p, y_train)
reg.score(X_test_p, y_test)

0.4184994762280899